# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words = text
    #set_words = set(words)
    count = Counter(words)
    sorted_vocab = sorted(count, key = count.get, reverse = True)
    vocab_to_int = {w:i for i,w in enumerate(sorted_vocab,1)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict={'.':'<PERIOD>',',':'<COMMA>', '"':'<QUOTATION_MARK>', ';': '<SEMICOLON>', '!': '<EXCLAMATION_MARK>', '?': '<QUESTION_MARK>', '(':'<LEFT_PAREN>', ')':'<RIGHT_PAREN>', '--':'<HYPHENS>', '\n':'<NEW_LINE>'}
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Inputs = tf.placeholder(tf.int32, shape = [None, None], name = 'input')
    Targets = tf.placeholder(tf.int32, shape = [None,None], name = 'targets')
    LearningRate = tf.placeholder(tf.float32, name = 'learning_rate')
    return Inputs, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm_layers = 1
    def build_cell(rnn_size):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        
        # Add dropout to the cell outputs
        #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return lstm
    # Stack up multiple LSTM layers, for deep learning
    Cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(lstm_layers)])
    # TODO: Implement Function
    
    #lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #Cell = tf.contrib.rnn.MultiRNNCell([lstm]*lstm_layers)
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState= tf.identity(InitialState, name= 'initial_state')
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform(shape=[vocab_size,embed_dim], minval = -1, maxval=1))
    embed_input= tf.nn.embedding_lookup(params = embedding, ids = input_data )
    return embed_input


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    out, finalstate = tf.nn.dynamic_rnn(cell, inputs = inputs, dtype = tf.float32)
    finalstate = tf.identity(finalstate, name = 'final_state')
    return out, finalstate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    input_embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, input_embed)
    logits = tf.contrib.layers.fully_connected(inputs= outputs, num_outputs = vocab_size)
    
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    #print(batch_size)
    #print(seq_length)
    number_batches = int(len(int_text)/(batch_size*seq_length))
    #print(len(int_text))
    #print(number_batches)
    batches = np.zeros((number_batches, 2, batch_size, seq_length), dtype = int)
    for i in range(number_batches):
        #input_batch = np.array([])
        #target_batch = np.array([])
        count = i*seq_length
        for j in range(batch_size):
            #print(count)
            if j == batch_size-1 and i == number_batches-1:
                batches[i][0][j]= np.array(int_text[count:count+seq_length])
                batches[i][1][j] = np.array(int_text[count+1:count+seq_length]+[int_text[0]])
                count = count + (number_batches*seq_length)
            else:
                batches[i][0][j]= np.array(int_text[count:count+seq_length])
                batches[i][1][j] = np.array(int_text[count+1:count+seq_length+1])
                count = count + (number_batches*seq_length)
                #print(batches[i][0][j])
            
            
            
        #batches[i][0] = input_batch
        #batches[i][1] = target_batch
    
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.0001
# Show stats for every n number of batches
show_every_n_batches = 50

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/69   train_loss = 8.819
Epoch   0 Batch   50/69   train_loss = 8.143
Epoch   1 Batch   31/69   train_loss = 6.788
Epoch   2 Batch   12/69   train_loss = 6.464
Epoch   2 Batch   62/69   train_loss = 6.339
Epoch   3 Batch   43/69   train_loss = 6.370
Epoch   4 Batch   24/69   train_loss = 6.218
Epoch   5 Batch    5/69   train_loss = 6.119
Epoch   5 Batch   55/69   train_loss = 6.070
Epoch   6 Batch   36/69   train_loss = 6.206
Epoch   7 Batch   17/69   train_loss = 6.145
Epoch   7 Batch   67/69   train_loss = 6.007
Epoch   8 Batch   48/69   train_loss = 6.164
Epoch   9 Batch   29/69   train_loss = 5.775
Epoch  10 Batch   10/69   train_loss = 5.943
Epoch  10 Batch   60/69   train_loss = 5.893
Epoch  11 Batch   41/69   train_loss = 5.797
Epoch  12 Batch   22/69   train_loss = 5.891
Epoch  13 Batch    3/69   train_loss = 5.600
Epoch  13 Batch   53/69   train_loss = 5.626
Epoch  14 Batch   34/69   train_loss = 5.714
Epoch  15 Batch   15/69   train_loss = 5.664
Epoch  15 

Epoch 132 Batch   42/69   train_loss = 3.997
Epoch 133 Batch   23/69   train_loss = 3.997
Epoch 134 Batch    4/69   train_loss = 4.071
Epoch 134 Batch   54/69   train_loss = 4.050
Epoch 135 Batch   35/69   train_loss = 4.247
Epoch 136 Batch   16/69   train_loss = 4.123
Epoch 136 Batch   66/69   train_loss = 4.084
Epoch 137 Batch   47/69   train_loss = 4.050
Epoch 138 Batch   28/69   train_loss = 3.858
Epoch 139 Batch    9/69   train_loss = 4.045
Epoch 139 Batch   59/69   train_loss = 3.987
Epoch 140 Batch   40/69   train_loss = 3.965
Epoch 141 Batch   21/69   train_loss = 3.925
Epoch 142 Batch    2/69   train_loss = 3.967
Epoch 142 Batch   52/69   train_loss = 3.747
Epoch 143 Batch   33/69   train_loss = 4.085
Epoch 144 Batch   14/69   train_loss = 3.899
Epoch 144 Batch   64/69   train_loss = 4.021
Epoch 145 Batch   45/69   train_loss = 4.045
Epoch 146 Batch   26/69   train_loss = 3.969
Epoch 147 Batch    7/69   train_loss = 3.918
Epoch 147 Batch   57/69   train_loss = 3.805
Epoch 148 

Epoch 265 Batch   15/69   train_loss = 3.154
Epoch 265 Batch   65/69   train_loss = 3.115
Epoch 266 Batch   46/69   train_loss = 3.171
Epoch 267 Batch   27/69   train_loss = 3.020
Epoch 268 Batch    8/69   train_loss = 2.953
Epoch 268 Batch   58/69   train_loss = 3.187
Epoch 269 Batch   39/69   train_loss = 3.092
Epoch 270 Batch   20/69   train_loss = 3.024
Epoch 271 Batch    1/69   train_loss = 2.827
Epoch 271 Batch   51/69   train_loss = 3.064
Epoch 272 Batch   32/69   train_loss = 2.861
Epoch 273 Batch   13/69   train_loss = 3.122
Epoch 273 Batch   63/69   train_loss = 3.005
Epoch 274 Batch   44/69   train_loss = 3.104
Epoch 275 Batch   25/69   train_loss = 3.062
Epoch 276 Batch    6/69   train_loss = 3.046
Epoch 276 Batch   56/69   train_loss = 3.151
Epoch 277 Batch   37/69   train_loss = 2.870
Epoch 278 Batch   18/69   train_loss = 2.918
Epoch 278 Batch   68/69   train_loss = 2.957
Epoch 279 Batch   49/69   train_loss = 2.826
Epoch 280 Batch   30/69   train_loss = 2.981
Epoch 281 

Epoch 397 Batch   57/69   train_loss = 2.155
Epoch 398 Batch   38/69   train_loss = 2.198
Epoch 399 Batch   19/69   train_loss = 2.195
Epoch 400 Batch    0/69   train_loss = 2.291
Epoch 400 Batch   50/69   train_loss = 2.408
Epoch 401 Batch   31/69   train_loss = 2.386
Epoch 402 Batch   12/69   train_loss = 2.182
Epoch 402 Batch   62/69   train_loss = 2.333
Epoch 403 Batch   43/69   train_loss = 2.395
Epoch 404 Batch   24/69   train_loss = 2.411
Epoch 405 Batch    5/69   train_loss = 2.286
Epoch 405 Batch   55/69   train_loss = 2.364
Epoch 406 Batch   36/69   train_loss = 2.430
Epoch 407 Batch   17/69   train_loss = 2.256
Epoch 407 Batch   67/69   train_loss = 2.304
Epoch 408 Batch   48/69   train_loss = 2.519
Epoch 409 Batch   29/69   train_loss = 2.360
Epoch 410 Batch   10/69   train_loss = 2.491
Epoch 410 Batch   60/69   train_loss = 2.376
Epoch 411 Batch   41/69   train_loss = 2.336
Epoch 412 Batch   22/69   train_loss = 2.280
Epoch 413 Batch    3/69   train_loss = 2.176
Epoch 413 

Epoch 530 Batch   30/69   train_loss = 2.050
Epoch 531 Batch   11/69   train_loss = 2.036
Epoch 531 Batch   61/69   train_loss = 2.069
Epoch 532 Batch   42/69   train_loss = 1.945
Epoch 533 Batch   23/69   train_loss = 1.835
Epoch 534 Batch    4/69   train_loss = 1.895
Epoch 534 Batch   54/69   train_loss = 1.956
Epoch 535 Batch   35/69   train_loss = 2.156
Epoch 536 Batch   16/69   train_loss = 2.081
Epoch 536 Batch   66/69   train_loss = 2.086
Epoch 537 Batch   47/69   train_loss = 2.110
Epoch 538 Batch   28/69   train_loss = 1.883
Epoch 539 Batch    9/69   train_loss = 2.011
Epoch 539 Batch   59/69   train_loss = 2.089
Epoch 540 Batch   40/69   train_loss = 1.973
Epoch 541 Batch   21/69   train_loss = 2.004
Epoch 542 Batch    2/69   train_loss = 1.888
Epoch 542 Batch   52/69   train_loss = 1.894
Epoch 543 Batch   33/69   train_loss = 2.048
Epoch 544 Batch   14/69   train_loss = 1.946
Epoch 544 Batch   64/69   train_loss = 2.014
Epoch 545 Batch   45/69   train_loss = 2.074
Epoch 546 

Epoch 663 Batch    3/69   train_loss = 1.871
Epoch 663 Batch   53/69   train_loss = 1.848
Epoch 664 Batch   34/69   train_loss = 2.016
Epoch 665 Batch   15/69   train_loss = 1.972
Epoch 665 Batch   65/69   train_loss = 1.986
Epoch 666 Batch   46/69   train_loss = 2.114
Epoch 667 Batch   27/69   train_loss = 1.919
Epoch 668 Batch    8/69   train_loss = 1.872
Epoch 668 Batch   58/69   train_loss = 2.183
Epoch 669 Batch   39/69   train_loss = 1.919
Epoch 670 Batch   20/69   train_loss = 1.966
Epoch 671 Batch    1/69   train_loss = 1.693
Epoch 671 Batch   51/69   train_loss = 2.126
Epoch 672 Batch   32/69   train_loss = 1.745
Epoch 673 Batch   13/69   train_loss = 2.147
Epoch 673 Batch   63/69   train_loss = 1.967
Epoch 674 Batch   44/69   train_loss = 2.026
Epoch 675 Batch   25/69   train_loss = 2.000
Epoch 676 Batch    6/69   train_loss = 2.006
Epoch 676 Batch   56/69   train_loss = 2.098
Epoch 677 Batch   37/69   train_loss = 1.823
Epoch 678 Batch   18/69   train_loss = 1.921
Epoch 678 

Epoch 795 Batch   45/69   train_loss = 1.964
Epoch 796 Batch   26/69   train_loss = 1.883
Epoch 797 Batch    7/69   train_loss = 1.819
Epoch 797 Batch   57/69   train_loss = 1.695
Epoch 798 Batch   38/69   train_loss = 1.794
Epoch 799 Batch   19/69   train_loss = 1.814
Epoch 800 Batch    0/69   train_loss = 1.886
Epoch 800 Batch   50/69   train_loss = 1.963
Epoch 801 Batch   31/69   train_loss = 1.919
Epoch 802 Batch   12/69   train_loss = 1.691
Epoch 802 Batch   62/69   train_loss = 1.904
Epoch 803 Batch   43/69   train_loss = 1.945
Epoch 804 Batch   24/69   train_loss = 2.035
Epoch 805 Batch    5/69   train_loss = 1.863
Epoch 805 Batch   55/69   train_loss = 1.967
Epoch 806 Batch   36/69   train_loss = 2.067
Epoch 807 Batch   17/69   train_loss = 1.920
Epoch 807 Batch   67/69   train_loss = 1.909
Epoch 808 Batch   48/69   train_loss = 2.160
Epoch 809 Batch   29/69   train_loss = 1.965
Epoch 810 Batch   10/69   train_loss = 2.138
Epoch 810 Batch   60/69   train_loss = 2.007
Epoch 811 

Epoch 928 Batch   18/69   train_loss = 1.886
Epoch 928 Batch   68/69   train_loss = 1.896
Epoch 929 Batch   49/69   train_loss = 1.833
Epoch 930 Batch   30/69   train_loss = 1.911
Epoch 931 Batch   11/69   train_loss = 1.915
Epoch 931 Batch   61/69   train_loss = 1.940
Epoch 932 Batch   42/69   train_loss = 1.782
Epoch 933 Batch   23/69   train_loss = 1.703
Epoch 934 Batch    4/69   train_loss = 1.777
Epoch 934 Batch   54/69   train_loss = 1.841
Epoch 935 Batch   35/69   train_loss = 2.029
Epoch 936 Batch   16/69   train_loss = 1.954
Epoch 936 Batch   66/69   train_loss = 1.944
Epoch 937 Batch   47/69   train_loss = 1.967
Epoch 938 Batch   28/69   train_loss = 1.752
Epoch 939 Batch    9/69   train_loss = 1.882
Epoch 939 Batch   59/69   train_loss = 1.954
Epoch 940 Batch   40/69   train_loss = 1.859
Epoch 941 Batch   21/69   train_loss = 1.882
Epoch 942 Batch    2/69   train_loss = 1.752
Epoch 942 Batch   52/69   train_loss = 1.764
Epoch 943 Batch   33/69   train_loss = 1.921
Epoch 944 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initialstate_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    finalstate_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initialstate_tensor, finalstate_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [24]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #print(probabilities)
    #print(type(probabilities))
    #print(int_to_vocab)
    #print(len(int_to_vocab))
    L = len(int_to_vocab)
    index = np.random.choice(L, 1, p=probabilities)
    index = int(index)
    predicted = int_to_vocab[index]
    return predicted


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [25]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: that's right-- and hasn't gumbo it wolfe.(theatah" fat_in_the_hat: the watchin' ah) what-- you do not?
barney_gumble: that, barney are nothin', homer. you're really othello what?
moe_szyslak: okay, i've lost.
homer_simpson: well, irishman...(trade remembered) hose cup?.. the you nuts soot. it was.
commit have, moonlight sail!
moe_szyslak: blues.(into phone) hello?! you can just lips again my gulliver_dark:.
yee-ha my name... complaint! / us back whispered sight-unseen a cell callin'
moe_szyslak: knees. from homer, in trainers in the your trade gang of jump.
homer_simpson: you are situation eighty-six before, aah i, listen you?
barney_gumble: so, i gotta frescas ran up the tree's a title.

kirk_van_houten: you can't my al_gore:.
moe_szyslak: oh, who i crowded am at me an raggie! the keep i dropped out fools(dejected_barfly:. powered my somethin') fine-lookin' planning drummer a trouble, who's diapers


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.